In [2]:
import numpy as np
import pandas as pd
import datetime as dt
pd.set_option('display.max_rows', 1000)
import hvplot.pandas
import holoviews as hv
from math import log
from bokeh.models import HoverTool
import warnings
import panel as pn

warnings.simplefilter('ignore')
pd.options.mode.chained_assignment = None  # default='warn'

**NEWS 2020-03-30:**  
- Added bar charts for latest per capita confirmed cases and deaths
- Added plots for Doubling Rates in Days **for U.S. States** 

<a id='top'></a>

**Examples Linear and Logarithmic Scale**  
[Confirmed Cases: USA](#Confirmed_Cases_USA)  
[Confirmed Cases: Germany](#Confirmed_Cases_Germany)  
**Latest Per Capita Confirmed Cases and Deaths by Country**  
[Latest per Capita Counts](#PerCapitaCasesDeathCountries)  
**Selected Countries Plots**  
[Confirmed Cases: Country Comparison](#Confirmed_Cases_Country_Comparison)  
[Deaths: Country Comparison](#Deaths_Country_Comparison)  
**Selected U.S. States Plots**  
[Confirmed Cases: U.S. States Comparison](#Confirmed_Cases_US_Comparison)  
[Deaths: U.S States Comparison](#Deaths_US_Comparison)  
**Other Plots**  
[Mortality](#Mortality)  
[3-day Change in Deaths](#ChangeDeaths)  
[3-day Change in Confirmed Cases](#ChangeConfirmedCases)  
NEW 2020-03-29: [Doubling Rate in Days: Confirmed Cases](#DoublingDaysConfirmedCases)  
NEW 2020-03-29: [Doubling Rate in Days: Deaths](#DoublingDaysDeaths)  
NEW 2020-03-30: [Doubling Rate in Days: Confirmed Cases (U.S.)](#DoublingDaysConfirmedCasesUS)  
NEW 2020-03-30: [Doubling Rate in Days: Deaths (U.S)](#DoublingDaysDeathsUS)  

[Suggested Reading](#Reading)  
[Data Sources](#DataSources)  

**NOTE: Move cursor over the plots to see actual data points**



# Daily Analysis of COVID-19 Data from Johns Hopkins University

In [3]:
print('\t\tUpdated every 6 hours. Last update:',dt.datetime.now().strftime('%Y-%m-%d %H:%M UTC'))

		Updated every 6 hours. Last update: 2020-03-31 02:20 UTC


Josef Kellndorfer, Ph.D., Earth Big Data, LLC, Richard Signell, Ph.D., USGS

These plots show the daily status of COVID-19 cases as reported by Johns Hopkins University. Please use freely to look at daily changes and trends. Keep in mind that data are changing frequently as more Covid-19 tests become available globally. We chose to plot **totals and** numbers **normalized by population** (expressed as per 100,000). Also, it is advantageous to plot case totals (confirmed infections, deaths, and recovered) in **logarithmic scale** where trends and parallels between countries become more obvious. Note, that a straight line trending upwards in logarithmic scale indicates exponential increase! Taking a close look at the plots, one will discern differences and similarities, and that for the most part initial stages are **similar in all countries with a time lag**. What to look out for is whether the measures taken by countries, foremost social distancing show the desired effects of slowing and eventually reversing the exponential upwards trends. The first set of plots looks at confirmed infections, the bottom set of plots looks at confirmed deaths, which may be somewhat more reliable with respect to an impact for a country while tests are rolled out in larger numbers. We also plot mortality (actual numbers and over time), and 3-day comparison of confirmend cases and deaths.

This is a work in progress, stay tuned. 

You can get the notebook underlying this work at:
[https://github.com/EarthBigData/covid19](https://github.com/EarthBigData/covid19)

For questions: Please use the email webform at [earthbigdata.com](http://earthbigdata.com)

## Interacting with the plots

You can use the control buttons to interact with the plots, e.g. **zoom in/out** or also **hover** over the data points **to get a detailed number**.

**Click on a label** in the legend **to dim/highlight a specific country or state**.

In [4]:
# World population
# pop='https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/CSV_FILES/WPP2019_TotalPopulationBySex.csv'
# We use a locally downloaded dataset from the link above
pop= '/s/data//WPP2019_TotalPopulationBySex.csv'

popdf = pd.read_csv(pop)
popdf.rename(columns={'Location': 'country','PopTotal':'population'}, inplace = True)

#popdf[(popdf.Time==2019) & (popdf.country=='China')].head(20)

popdf2=popdf[popdf.Time==2019]
#[ x for x in popdf2.country.values if x.find('Korea') > -1]

# Necessary adjustmens of names so we can link the population and jhu data sets
popdf2.loc[popdf2.country=='United States of America','country']='US'
popdf2.loc[popdf2.country=='Iran (Islamic Republic of)','country']='Iran'
popdf2.loc[popdf2.country=='Republic of Korea','country']='Korea, South'

In [5]:
# US Population
#popus='http://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv?#'
popus='/s/data/nst-est2019-alldata.csv'
popusdf = pd.read_csv(popus)
popusdf['POPESTIMATE2019']
popusdf.rename(columns={'NAME': 'state','POPESTIMATE2019':'population'}, inplace = True)

In [6]:
# Sources for COVID-19 
# c = confirmed cases
# d = deaths
# r = recovered

cv19_c ='https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
cv19_d ='https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
#old
#cv19_co ='https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
# cv19_do ='https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
# Switching for now to data for US provided in old format from soothsawyer
# cv19_co ='https://www.soothsawyer.com/john-hopkins-time-series-data-confirmed-case-csv-after-march-22-2020/?github=1'
# cv19_do ='https://www.soothsawyer.com/wp-content/uploads/2020/03/time_series_19-covid-Deaths.csv'
cv19_co='/s/data/covid19/time_series_19-covid-Confirmed.csv'
cv19_do='/s/data/covid19/time_series_19-covid-Deaths.csv'

# cv19_r ='https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'

In [7]:
# Pre-Selected Countries
countries=('China','Korea, South','Japan','Italy','Iran','Spain','Germany',
           'Austria','France','Sweden','United Kingdom','US','Mexico','Canada',
           'Brazil','India','Australia','South Africa','Egypt'
          )

In [8]:
# Pre-Selected States
states=(('Massachusetts','MA'),('Connecticut','CT'),('Washington','WA'),
        ('Texas','TX'),('California','CA'),('New York','NY'),
        ('Georgia','GA'),('Illinois','IL'),('Michigan','MI'),
        ('Louisiana','LA'),('Florida','FL'),('New Jersey','NJ'),
        ('Colorado','CO'),('Indiana','IN')
       )

<a id='Confirmed_Cases_USA'></a>
## Confirmed Cases: Status for United States
[back to top](#top)

In [9]:
df = pd.read_csv(cv19_c,na_values=0)
df.fillna(0,inplace=True)
df.iloc[:,5:] = df.iloc[:,5:].astype(int)

df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))

df = df[(df.country=='US') & (df.state!='Diamond Princess') & 
        (df.state!='Grand Princess')].reset_index(drop=True)

logscale=False
title='Confirmed cases of COVID-19 in the USA. Status: {}'.format(df.columns[-1].date())

s = df.select_dtypes(np.int).sum()
upperlim=s[-1]*1.2
ylim=(1,upperlim)
s.name = 'USA'

logscale=False
title='Confirmed cases of COVID-19 in {}, Status: {}'.format(s.name,df.columns[-1].date())
opts = {'legend':True, 'logy': logscale, 'grid': True,'title': title, 'padding':0.1,'ylim':ylim,
        'ylabel':'Number of Confirmed Cases','xlabel':'Date','width':700}
lines = s.hvplot(**opts) 
dots  = s.hvplot.scatter(**opts)
layoutlin = (lines * dots).opts(active_tools=['box_zoom'])

logscale=True
title='Confirmed cases of COVID-19 in {}, Status: {}, Logarithmic Scale'.format(s.name,df.columns[-1].date())
opts = {'legend':True, 'logy': logscale, 'grid': True,'title': title, 'padding':0.1,'ylim':ylim,
        'ylabel':'Number of Confirmed Cases','xlabel':'Date','width':700}
lines = s.hvplot(**opts) 
dots  = s.hvplot.scatter(**opts)
layoutlog = (lines * dots).opts(active_tools=['box_zoom'])

layout=hv.Layout([layoutlin,layoutlog]).opts(shared_axes=False,merge_tools=False)
#layout.toolbar.active_drag = None
layout.cols(1)

:Layout
   .Overlay.USA.I  :Overlay
      .Curve.USA   :Curve   [index]   (USA)
      .Scatter.USA :Scatter   [index]   (USA)
   .Overlay.USA.II :Overlay
      .Curve.USA   :Curve   [index]   (USA)
      .Scatter.USA :Scatter   [index]   (USA)

<a id='Confirmed_Cases_Germany'></a>
## Confirmed Cases: Status for Germany
[back to top](#top)

In [58]:
df = pd.read_csv(cv19_c,na_values=0)
df.fillna(0,inplace=True)
df.iloc[:,5:] = df.iloc[:,5:].astype(int)

df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))

df.state='Germany'
df=df[(df.country=='Germany')].reset_index(drop=True)

s = df.select_dtypes(np.int).sum()
upperlim=s[-1]*1.2
ylim=(1,upperlim)

s.name = 'Germany'

logscale=False
title='Confirmed cases of COVID-19 in Germany, Status: {}'.format(df.columns[-1].date())
# opts = {'legend':True, 'logy': logscale, 'grid': True, 'width': 350, 'height': 400,
#         'title': title, 'padding':0.1,'ylim':ylim,'ylabel':'Number of Confirmed Cases','xlabel':'Date'}
opts = {'legend':True, 'logy': logscale, 'grid': True,'title': title, 'padding':0.1,'ylim':ylim,
        'ylabel':'Number of Confirmed Cases','xlabel':'Date','width':700}
lines = s.hvplot(**opts) 
dots  = s.hvplot.scatter(**opts)
layoutlin = (lines * dots).opts(active_tools=['box_zoom'])

logscale=True
title='Confirmed cases of COVID-19 in Germany, Status: {}, Logarithmic Scale'.format(df.columns[-1].date())
# opts = {'legend':True, 'logy': logscale, 'grid': True, 'width': 350, 'height': 400,
#         'title': title, 'padding':0.1,'ylim':ylim,'ylabel':'Number of Confirmed Cases','xlabel':'Date'}
opts = {'legend':True, 'logy': logscale, 'grid': True,'title': title, 'padding':0.1,'ylim':ylim,
        'ylabel':'Number of Confirmed Cases','xlabel':'Date','width':700}
lines = s.hvplot(**opts) 
dots  = s.hvplot.scatter(**opts)
layoutlog = (lines * dots).opts(active_tools=['box_zoom'])

layout=hv.Layout([layoutlin,layoutlog]).opts(shared_axes=False,merge_tools=False)
layout.cols(1)

:Layout
   .Overlay.Germany.I  :Overlay
      .Curve.Germany   :Curve   [index]   (Germany)
      .Scatter.Germany :Scatter   [index]   (Germany)
   .Overlay.Germany.II :Overlay
      .Curve.Germany   :Curve   [index]   (Germany)
      .Scatter.Germany :Scatter   [index]   (Germany)

<a id='PerCapitaCasesDeathCountries'></a>
##  Latest Confirmed Cases and Deaths Per Capita: Country Comparison
[back to top](#top)

In [ ]:
pdf=popdf2
# Deaths
df_d = pd.read_csv(cv19_d)
df_d.columns = df_d.columns[0:4].append(pd.to_datetime(df_d.columns[4:]))
df_d.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
# Confirmed cases
df_c = pd.read_csv(cv19_c)
df_c.columns = df_c.columns[0:4].append(pd.to_datetime(df_c.columns[4:]))
df_c.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
# Population
pdf=pdf[pdf.Time==2019]
pdf.set_index(pdf.country,inplace=True)
pdf=pdf[pdf.Time==2019].loc[list(countries)].sort_index().population

# Get the latest numbers
latest_c = [df_c.columns[1],df_c.columns[-1]]
df_c = df_c[latest_c].groupby('country').sum()
latest_d = [df_d.columns[1],df_d.columns[-1]]
df_d = df_d[latest_d].groupby('country').sum()

# Compute per capita numbers
c = df_c.select_dtypes(np.int).loc[list(countries)].sort_index().iloc[:,0]
d = df_d.select_dtypes(np.int).loc[list(countries)].sort_index().iloc[:,0]

# Cases
m=c/pdf*100
m.dropna(inplace=True)
m.sort_values(ascending=True,inplace=True)
title='COVID-19 Confirmed cases per 100,000 by Country, Status {}'.format(latest_c[-1].date())
cases = m.hvplot.barh(padding=0.1,grid=True,ylabel='Confirmed Cases per Capita [per 100,000]',xlabel='',title=title)
# Deaths
m=d/pdf*1000
m.dropna(inplace=True)
m.sort_values(ascending=True,inplace=True)
title='COVID-19 Deaths per 1,000,000 by Country, Status {}'.format(latest_c[-1].date())
deaths = m.hvplot.barh(padding=0.1,grid=True,ylabel='Deaths per Capita [per 1,000,000]',xlabel='',title=title)

layout=hv.Layout([cases,deaths])
layout.cols(1)

<a id='Confirmed_Cases_Country_Comparison'></a>
##  Confirmed Cases: Country Comparison
[back to top](#top)

In [ ]:
#countries=('China','Korea, South','Japan','Italy','Iran','Spain','Germany','Austria','France','Sweden','United Kingdom','US','Mexico','Canada','Brazil','India','Australia')
df = pd.read_csv(cv19_c,na_values=0)
df.fillna(0,inplace=True)
df.iloc[:,5:] = df.iloc[:,5:].astype(int)
df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))

df = df[[x in countries for x in df.country.values]]

def uplim(df,pdf,normalize=False,extend=1.2):  
    smax=0
    for i in df.country:
        s = df[df.country==i].select_dtypes(np.int).sum()[-1]
        if normalize:
            s = s/(pdf[(pdf.country==i) & (pdf.Time==2019)].population.values[0]*1000)*100000
        if s > smax:
            smax=s   
    return smax*extend,log(smax*extend) 

def country(name='Germany',normalize_by_population=False,logy=False):
    country = df[(df.country==name)]
    s = country.select_dtypes(np.int).sum()
    if normalize_by_population:
        ulim,ulimlog=uplim(country,popdf2)
        s = s/(popdf2[(popdf2.country==name) & (popdf2.Time==2019)].population.values[0]*1000)*100000
#         upperlim=s[-1]*1.2
#         ylim=(0.00001,upperlim)
#         print(ylim)
#     else:
#         upperlim=s[-1]*1.2
#         ylim=(1,upperlim)
#     opts = {'legend': True, 'logy': logy, 'grid': True, 'width': 700, 'height': 400,
#         'title': f'Confirmed cases of COVID-19 in {name}', 'padding':0.1,
#         'ylim':ylim}
    opts = {'legend': True, 'logy': logy, 'grid': True, 'width': 700, 'height': 400,'padding':0.1,'size':10}
    s.name = name
    lines = s.hvplot(**opts)
    dots = s.hvplot.scatter(**opts)
    hstate = (lines * dots)
    return hstate

ulim,ulimlog=uplim(df,popdf2)
nulim,nulimlog=uplim(df,popdf2,True)

lo=[]
for normalize_by_population in [False,True]:
    for logscale in [False, True]:
    
        if normalize_by_population:
            ylabel='Number of cases normalized by population [per 100,000]'
            title_format='COVID-19 Confirmed Cases: Normalized by Population, Status: {}'.format(df.columns[-1].date())
            ylim=(0.00001,nulim)
        else:
            ylim=(1,ulim)
            ylabel='Total Number of Cases [Count]'
            title_format='COVID-19 Confirmed Cases: Total Count, Status: {}'.format(df.columns[-1].date())

        if logscale:
            title_format+=', Logarithmic Scale'

        ol=[]
        for i in countries:
            ol.append(country(i,normalize_by_population,logscale))

        overlay=hv.Overlay(ol)
        lo.append(overlay.opts(height=600,width=780,title_format=title_format,ylabel=ylabel,
                               xlabel='Date',xrotation=45,ylim=ylim,logy=logscale,active_tools=['box_zoom'],toolbar='right'))
layout=hv.Layout(lo).opts(shared_axes=False,merge_tools=False)
layout.cols(1)

<a id='Deaths_Country_Comparison'></a>
## Deaths Country Comparison
[back to top](#top)

In [ ]:
#countries=('China','Korea, South','Japan','Italy','Iran','Spain','Germany','Austria','France','US','Mexico','Canada','Brazil','India','United Kingdom')

df = pd.read_csv(cv19_d,na_values=0)
df.fillna(0,inplace=True)
df.iloc[:,5:] = df.iloc[:,5:].astype(int)

df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))

df = df[[x in countries for x in df.country.values]]

def uplim(df,pdf,normalize=False,extend=1.2):  
    smax=0
    for i in df.country:
        s = df[df.country==i].select_dtypes(np.int).sum()[-1]
        if normalize:
            s = s/(pdf[(pdf.country==i) & (pdf.Time==2019)].population.values[0]*1000)*100000
        if s > smax:
            smax=s   
    return smax*extend,log(smax*extend) 

def country(name='Germany',normalize_by_population=False,logy=False):
    country = df[(df.country==name)]
    s = country.select_dtypes(np.int).sum()
    if normalize_by_population:
        ulim,ulimlog=uplim(country,popdf2)
        s = s/(popdf2[(popdf2.country==name) & (popdf2.Time==2019)].population.values[0]*1000)*100000
#         upperlim=s[-1]*1.2
#         ylim=(0.00001,upperlim)
#         print(ylim)
#     else:
#         upperlim=s[-1]*1.2
#         ylim=(1,upperlim)
#     opts = {'legend': True, 'logy': logy, 'grid': True, 'width': 700, 'height': 400,
#         'title': f'Confirmed cases of COVID-19 in {name}', 'padding':0.1,
#         'ylim':ylim}
    opts = {'legend': True, 'logy': logy, 'grid': True, 'width': 700, 'height': 400,'padding':0.1,'size':10}
    s.name = name
    lines = s.hvplot(**opts)
    dots = s.hvplot.scatter(**opts)
    hstate = lines * dots
    return hstate

ulim,ulimlog=uplim(df,popdf2)
nulim,nulimlog=uplim(df,popdf2,True)

lo=[]
for normalize_by_population in [False,True]:
    for logscale in [False, True]:
    
        if normalize_by_population:
            ylabel='Number of Deaths normalized by population [per 100,000]'
            title_format='COVID-19 Deaths: Normalized by Population, Status: {}'.format(df.columns[-1].date())
            ylim=(0.00001,nulim)
        else:
            ylim=(1,ulim)
            ylabel='Total Number of Deaths [Count]'
            title_format='COVID-19 Deaths: Total Count, Status: {}'.format(df.columns[-1].date())

        if logscale:
            title_format+=', Logarithmic Scale'

        ol=[]
        for i in countries:
            ol.append(country(i,normalize_by_population,logscale))

        overlay=hv.Overlay(ol)
        lo.append(overlay.opts(height=600,width=780,title_format=title_format,ylabel=ylabel,
                               xlabel='Date',xrotation=45,ylim=ylim,logy=logscale,active_tools=['box_zoom'],toolbar='right'))
layout=hv.Layout(lo).opts(shared_axes=False,merge_tools=False)
layout.cols(1)

<a id='Confirmed_Cases_US_Comparison'></a>
## Confirmed Cases: U.S. States Comparison
[back to top](#top)

In [ ]:
# states=(('Massachusetts','MA'),('Connecticut','CT'),('Washington','WA'),
#         ('Texas','TX'),('California','CA'),('New York','NY'),
#         ('Georgia','GA'),('Illinois','IL'),('Michigan','MI'),
#         ('Louisiana','LA'),('Florida','FL')
#        )

df = pd.read_csv(cv19_co)
df.fillna(0,inplace=True)
df.iloc[:,5:] = df.iloc[:,5:].astype(int)
df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df = df[(df.country=='US') & (df.state!='Diamond Princess') & 
        (df.state!='Grand Princess')].reset_index(drop=True)
df.columns = df.columns[0:4].append(pd.to_datetime(pd.to_datetime(df.columns[4:]).date))
state = df.state.str.split(',').apply(lambda x: x[-1].strip())
county = df.state.str.split(',').apply(lambda x: x[0].strip())
county[~df.state.str.contains(',')] = None
df.state = state
df.insert(0, 'county', county)


snames=[x[0] for x in states]
scodes=[x[1] for x in states]
df = df[[x in snames or x in scodes for x in df.state.values]]

def uplim(states,df,pdf,normalize=False,extend=1.2):  
    smax=0
    for i in states:
        name,code=i
        state = df[(df.state==name) | (df.state==code)]
        s = state.select_dtypes(np.int).sum()[-1]
        if normalize:
            s = s/pdf[pdf.state==name].population.values[0]*100000
        if s > smax:
            smax=s   
    return smax*extend,log(smax*extend) 

def state(name='Massachusetts',code='MA',normalize_by_population=False,logy=False):
    state = df[(df.state==name) | (df.state==code)]
    s = state.select_dtypes(np.int).sum()
    if normalize_by_population:
        s = s/popusdf[popusdf.state==name].population.values[0]*100000
    s.name = name  
#     opts = {'legend': True, 'logy': logy, 'grid': True, 'width': 700, 'height': 400,'padding':0.1,'size':10,'tools':[]}
#     lines = s.hvplot(**opts).opts(tools=[])
#     hover = HoverTool(tooltips=[('Date', '$x{%F}'),(name,'$y{0}')], formatters={'$x': 'datetime'}, mode="mouse")
#     opts = {'legend': True, 'logy': logy, 'grid': True, 'width': 700, 'height': 400,'padding':0.1,'size':10,'tools':[hover]}
#     dots = s.hvplot.scatter(**opts)

    hover = HoverTool(tooltips=[('Date', '$x{%F}'),(name,'$y{0}')], formatters={'$x': 'datetime'}, mode="mouse")
    opts = {'legend': True, 'logy': logy, 'grid': True, 'width': 700, 'height': 400,'padding':0.1,'size':10,'tools':[hover]}
    lines = s.hvplot(**opts).opts(tools=[])
    dots = s.hvplot.scatter(**opts)
    hstate = lines * dots
    return hstate

sd='2020-02-15'
ed=(df.columns[-1]+dt.timedelta(1)).strftime('%Y-%m-%d')
xlim=(pd.date_range(sd,ed)[0],pd.date_range(sd,ed)[-1])

ulim,ulimlog=uplim(states,df,popusdf)
nulim,nulimlog=uplim(states,df,popusdf,True)

lo=[]
for normalize_by_population in [False,True]:
    for logscale in [False, True]:
    
        if normalize_by_population:
            ylim=(0.001,nulim)
            ylabel='Number of cases normalized by population [per 100,000]'
            title_format='U.S. COVID-19 Confirmed Cases: Normalized by Population, Status: {}'.format(df.columns[-1].date())
            #title_format='U.S. COVID-19 Confirmed Cases: Normalized by Population, Status: 2020-03-22'
        else:
            ylim=(1,ulim)
            ylabel='U.S. Total Number of Cases [Count]'
            title_format='U.S. COVID-19 Confirmed Cases: Total Count, Status: {}'.format(df.columns[-1].date())
            #title_format='U.S. COVID-19 Confirmed Cases: Total Count, Status: 2020-03-22'
        ol=[]
        for i in states:
            ol.append(state(*i,normalize_by_population,logscale))

        if logscale:
            title_format+=', Log. Scale'

        overlay=hv.Overlay(ol)

        

        lo.append(overlay.opts(height=600,width=780,title_format=title_format, 
                               ylim=ylim,xlim=xlim,ylabel=ylabel,xlabel='Date',
                               xrotation=45,active_tools=['box_zoom'],toolbar='right'))
layout=hv.Layout(lo).opts(shared_axes=False,merge_tools=False)
layout.cols(1)

<a id='Deaths_US_Comparison'></a>
## U.S.Deaths
[back to top](#top)

In [ ]:
#states=(('Massachusetts','MA'),('Connecticut','CT'),('Washington','WA'),('Texas','TX'),('California','CA'),('New York','NY'),('Georgia','GA'),('Illinois','IL'),('Michigan','MI'))

df = pd.read_csv(cv19_do)
df.fillna(0,inplace=True)
df.iloc[:,5:] = df.iloc[:,5:].astype(int)
df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df = df[(df.country=='US') & (df.state!='Diamond Princess') & 
        (df.state!='Grand Princess')].reset_index(drop=True)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))
state = df.state.str.split(',').apply(lambda x: x[-1].strip())
county = df.state.str.split(',').apply(lambda x: x[0].strip())
county[~df.state.str.contains(',')] = None
df.state = state
df.insert(0, 'county', county)


snames=[x[0] for x in states]
scodes=[x[1] for x in states]
df = df[[x in snames or x in scodes for x in df.state.values]]

def uplim(states,df,pdf,normalize=False,extend=1.2):  
    smax=0
    for i in states:
        name,code=i
        state = df[(df.state==name) | (df.state==code)]
        s = state.select_dtypes(np.int).sum()[-1]
        if normalize:
            s = s/pdf[pdf.state==name].population.values[0]*100000
        if s > smax:
            smax=s   
    return smax*extend,log(smax*extend) 

def state(name='Massachusetts',code='MA',normalize_by_population=False,logy=False):
    state = df[(df.state==name) | (df.state==code)]
    s = state.select_dtypes(np.int).sum()
    if normalize_by_population:
        s = s/popusdf[popusdf.state==name].population.values[0]*100000
    opts = {'legend': True, 'logy': logy, 'grid': True, 'width': 700, 'height': 400,'padding':0.1,'size':10}
    s.name = name
    lines = s.hvplot(**opts)
    dots = s.hvplot.scatter(**opts)
    hstate = lines * dots
    return hstate

sd='2020-03-01'
ed=(df.columns[-1]+dt.timedelta(1)).strftime('%Y-%m-%d')
# xlim=(pd.date_range(sd,ed)[0],pd.date_range(sd,ed)[-1])

ulim,ulimlog=uplim(states,df,popusdf)
nulim,nulimlog=uplim(states,df,popusdf,True)

lo=[]
for normalize_by_population in [False,True]:
    for logscale in [False, True]:
    
        if normalize_by_population:
            ylim=(0.001,nulim)
            ylabel='Number of Deaths normalized by population [per 100,000]'
            title_format='U.S. COVID-19 Deaths: Normalized by Population, Status: {}'.format(df.columns[-1].date())
            #title_format='U.S. COVID-19 Deaths: Normalized by Population, Status: 2020-03-22'
        else:
            ylim=(1,ulim)
            ylabel='U.S. Total Number of Deaths [Count]'
            title_format='U.S. COVID-19 Deaths: Total Count, Status: {}'.format(df.columns[-1].date())
            #title_format='U.S. COVID-19 Deaths: Total Count, Status: 2020-03-22'
        ol=[]
        for i in states:
            ol.append(state(*i,normalize_by_population,logscale))

        if logscale:
            title_format+=', Logarithmic Scale'

        overlay=hv.Overlay(ol)
        lo.append(overlay.opts(height=600,width=780,title_format=title_format, ylim=ylim,xlim=xlim,
                               ylabel=ylabel,xlabel='Date',xrotation=45,active_tools=['box_zoom'],toolbar='right'))
layout=hv.Layout(lo).opts(shared_axes=False,merge_tools=False)
layout.cols(1)

<a id='Mortality'></a>
## Mortality Rates Country Comparison
[back to top](#top)

The plot below shows the *mortality rate* in percent computed as:

$Mortality=\frac{Deaths}{Infected} * 100$

Two caveats:
- Because there is a timelag from infection to death, the rates computed on a daily basis might be underestimating the rate.
- The rates might be vastly overestimated because the lag in testing is widespread. South Korea has done the most intensive testing, hence South Korea data seem most reliable for mortality assessment.

In [ ]:
df_d = pd.read_csv(cv19_d)
df_d.columns = df_d.columns[0:4].append(pd.to_datetime(df_d.columns[4:]))
df_d.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
#df_d = df_d[[x in countries for x in df_d.country.values]]
# Infections confirmed cases
df_c = pd.read_csv(cv19_c)
df_c.columns = df_c.columns[0:4].append(pd.to_datetime(df_c.columns[4:]))
df_c.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
#df_c = df_c[[x in countries for x in df_c.country.values]]


latest = [df_d.columns[1],df_d.columns[-1]]
df_d = df_d[latest].groupby('country').sum()
df_c = df_c[latest].groupby('country').sum()

c = df_c.select_dtypes(np.int)
d = df_d.select_dtypes(np.int)
m=d/c*100
m.dropna(inplace=True)
#m.sort_values(m.columns[0],ascending=False,inplace=True)

m_selected = m.loc[countries,:]

m_selected.sort_values(m_selected.columns[0],ascending=True,inplace=True)

title='Mortality by Country, Status {} (See Caveats!)'.format(m.columns[0].date())
m_selected.hvplot.barh(padding=0.1,grid=True,ylabel='Mortality [%]',xlabel='',title=title)

In [ ]:
#countries=('China','Korea, South','Japan','Italy','Iran','Spain','Germany','Austria','France','US','Mexico','Canada','Brazil','India','United Kingdom')

#Deaths
df_d = pd.read_csv(cv19_d)
df_d.columns = df_d.columns[0:4].append(pd.to_datetime(df_d.columns[4:]))
df_d.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df_d = df_d[[x in countries for x in df_d.country.values]]
# Infections confirmed cases
df_c = pd.read_csv(cv19_c)
df_c.columns = df_c.columns[0:4].append(pd.to_datetime(df_c.columns[4:]))
df_c.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df_c = df_c[[x in countries for x in df_c.country.values]]


def uplim(df,pdf,normalize=False,extend=1.2):  
    smax=0
    for i in df.country:
        s = df[df.country==i].select_dtypes(np.int).sum()[-1]
        if normalize:
            s = s/(pdf[(pdf.country==i) & (pdf.Time==2019)].population.values[0]*10)
        if s > smax:
            smax=s   
    return smax*extend,log(smax*extend) 

def mortality(name='Germany',normalize_by_population=False,logy=False):
    country_deaths   = df_d[(df_d.country==name)]
    country_infected = df_c[(df_c.country==name)]
    d = country_deaths.select_dtypes(np.int).sum()
    i = country_infected.select_dtypes(np.int).sum()
    m=d/i*100
    if normalize_by_population:
        ulim,ulimlog=uplim(country,popdf2)
        m = m/(popdf2[(popdf2.country==name) & (popdf2.Time==2019)].population.values[0]*10)
    opts = {'legend': True, 'logy': logy, 'grid': True, 'width': 700, 'height': 400,'padding':0.1,'size':10}
    m.name = name
    lines = m.hvplot(**opts)
    dots = m.hvplot.scatter(**opts)
    hstate = lines * dots
    return hstate

ylim=(-1,15)
ylabel='Mortality [%]'
title_format='COVID-19 Mortality (See Caveats!)'
normalize_by_population=False
logscale=False
ol=[]
for i in countries:
    ol.append(mortality(i,normalize_by_population,logscale))

overlay=hv.Overlay(ol).opts(height=600,width=780,title_format=title_format,ylabel=ylabel,
                            xlabel='Date',xrotation=45,ylim=ylim,logy=logscale,
                            active_tools=['box_zoom'],toolbar='right')
overlay

<a id='ChangeDeaths'></a>
## 3-Day Change in Deaths: Country Comparison
[back to top](#top)

The plots below show the  change of **total number of deaths compared to three days before the plotted date**. A factor **2 means the cases doubled** after three days. A factor **1 means no new deaths are reported** compared to three days before. (Plots also inspired by Jennifer Bardwell, Jim Bardwell).


In [ ]:
df = pd.read_csv(cv19_d,na_values=0)
df.fillna(0,inplace=True)
df.iloc[:,5:] = df.iloc[:,5:].astype(int)

df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))

df = df[[x in countries for x in df.country.values]]

def country(name='Germany',normalize_by_population=False,logy=False):
    country = df[(df.country==name)]
    s = country.select_dtypes(np.int).sum()
    d=s[2:]
    r= s[2:].values/s[:-2].values
#     new=s[3:].values-s[1:-2].values
#     previous=s[2:].values-s[:-3].values
#     r= new.values/previous.values*100

    d[:]=r    
    opts = {'ylim':(1,4),'legend': False, 'logy': logy, 'grid': True, 'width': 320, 'height': 200,'padding':0.1,'size':10}
    d.name = name
    lines = d.hvplot(**opts)
    dots = d.hvplot.scatter(**opts)
    hline200=hv.HLine(2).opts(color='red', line_width=1)
    hline100=hv.HLine(1).opts(color='black', line_width=1)
    hstate = (lines * dots  *hline200 * hline100)
    return hstate

ylim=(1,4)
ylabel='3-day Death increase Factor'
title_format='COVID-19 3-Day Death Increase Factor: {}'.format(df.columns[-1].date())
ol=[]
for i in countries:
    ol.append(country(i))

overlay=hv.Layout(ol)
overlay.opts(
    merge_tools=True,title=title_format)
overlay.cols(2)


<a id='ChangeConfirmedCases'></a>
## 3-Day Change in Confirmed Cases: Country Comparison
[back to top](#top)

The plots below show the  change of **total number of confirmed cases compared to three days before the plotted date**. A factor **2 means the cases doubled** after three days. A factor **1 means no new confirmed cases are reported** compared to three days before. (Plots also inspired by Jennifer Bardwell, Jim Bardwell).


In [ ]:
df = pd.read_csv(cv19_c,na_values=0)
df.fillna(0,inplace=True)
df.iloc[:,5:] = df.iloc[:,5:].astype(int)

df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))

df = df[[x in countries for x in df.country.values]]

def country(name='Germany',normalize_by_population=False,logy=False):
    country = df[(df.country==name)]
    s = country.select_dtypes(np.int).sum()
    d=s[2:]
    r= s[2:].values/s[:-2].values
#     new=s[3:].values-s[1:-2].values
#     previous=s[2:].values-s[:-3].values
#     r= new.values/previous.values*100

    d[:]=r    
    opts = {'ylim':(1,4),'legend': False, 'logy': logy, 'grid': True, 'width': 320, 'height': 200,'padding':0.1,'size':10}
    d.name = name
    lines = d.hvplot(**opts)
    dots = d.hvplot.scatter(**opts)
    hline200=hv.HLine(2).opts(color='red', line_width=1)
    hline100=hv.HLine(1).opts(color='black', line_width=1)
    hstate = (lines * dots  *hline200 * hline100)
    return hstate

ylim=(1,4)
ylabel='3-day confirmed cases increase Factor'
title_format='COVID-19 3-Day Confirmed Cases Increase Factor: {}'.format(df.columns[-1].date())
ol=[]
for i in countries:
    ol.append(country(i))

overlay=hv.Layout(ol)
overlay.opts(
    merge_tools=True,title=title_format)
overlay.cols(2)


<a id='DoublingDaysConfirmedCases'></a>
## Number of Days to Double Confirmed Cases: Country Comparison
[back to top](#top)

The plots below show the change rate (${doubling.rate}_{confirmed.cases}$) in **number of days for confirmed cases to double** ($days_{confirmed.cases.double}$). This is expressed as

${doubling.rate}_{confirmed.cases} = \frac{1}{days_{confirmed.cases.double}}$

In this representation a factor of 1 means cases double every day, 0.5 means cases double every 2nd day, 0.33 means cases double every third daty, 0.25 menas cases double every 4th day, etc. When the line approaches 0, no more cases are identified.  
Plots begin at **more than 100 confirmed cases**.   
(Plots inspired by Jennifer Bardwell, Jim Bardwell).


In [ ]:
df = pd.read_csv(cv19_c,na_values=0)
df.fillna(0,inplace=True)
df.iloc[:,5:] = df.iloc[:,5:].astype(int)

df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))

df = df[[x in countries for x in df.country.values]]

def double_rate(name,logy=False):
    country = df[(df.country==name)]
    s = country.select_dtypes(np.int).sum()
    r = pd.Series([np.nan for x in range(len(s.values))],index=s.index)
    for x in range(len(s.values)-1,0,-1):
        if s[x]>100:   # We start after 100 cases has been identified
            half=s[x]/2
            mask=[s < half][0] 
            half_idx = mask.values.argmin()
            frac_day=1-(half-s[half_idx-1])/(s[half_idx]-s[half_idx-1])
            doubling_days=x-half_idx+frac_day
            r[x]=1/doubling_days

    d=r    
    opts = {'ylim':(0,1),'legend': False, 'logy': logy, 'grid': True, 'width': 320, 'height': 200,'padding':0.1,'size':10}
    d.name = name
    lines = d.hvplot(**opts)
    dots = d.hvplot.scatter(**opts)
    hline2=hv.HLine(0.5).opts(color='red', line_width=1,line_dash='solid')
    hline3=hv.HLine(0.33).opts(color='red', line_width=1,line_dash='dashed')
    hline5=hv.HLine(0.2).opts(color='red', line_width=1,line_dash='dashdot')
    hline10=hv.HLine(0.1).opts(color='red', line_width=1,line_dash='dotted')
    text2d = hv.Text(s.index[1],0.53,'2 days',fontsize=8)
    text3d = hv.Text(s.index[1],0.36,'3 days',fontsize=8)
    text5d = hv.Text(s.index[1],0.23,'5 days',fontsize=8)
    text10d = hv.Text(s.index[1],0.13,'10 days',fontsize=8)
    hstate = (lines * dots  *hline2 * hline3 * hline5 *hline10 * text2d * text3d * text5d * text10d).opts(show_legend=True)
    return hstate

ylim=(0,1)
ylabel='Inverse Number of days for Confirmed Cases to Double'
title_format='COVID-19 Days for Confirmed Cases to Double: {}'.format(df.columns[-1].date())
ol=[]
for i in countries:
    ol.append(double_rate(i))

overlay=hv.Layout(ol)
overlay.opts(
    merge_tools=True,title=title_format)
overlay.cols(2)


<a id='DoublingDaysDeaths'></a>
## Number of Days to Double Deaths: Country Comparison
[back to top](#top)

The plots below show the change rate (${doubling.rate}_{deaths}$) in **number of days for deaths to double** ($days_{deaths.double}$). This is expressed as

${doubling.rate}_{deaths} = \frac{1}{days_{deaths.double}}$

In this representation a factor of 1 means death counts double every day, 0.5 means death counts double every 2nd day, 0.33 means death counts double every third day, 0.25 menas death counts double every 4th day, etc. When the line approaches 0, no more deaths are counted.  
Plots begin at **more than 25 deaths**.  
*(Plots inspired by Jennifer Bardwell, Jim Bardwell).*


In [ ]:
df = pd.read_csv(cv19_d,na_values=0)
df.fillna(0,inplace=True)
df.iloc[:,5:] = df.iloc[:,5:].astype(int)

df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))

df = df[[x in countries for x in df.country.values]]

def double_rate(name,logy=False):
    country = df[(df.country==name)]
    s = country.select_dtypes(np.int).sum()
    r = pd.Series([np.nan for x in range(len(s.values))],index=s.index)
    for x in range(len(s.values)-1,0,-1):
        if s[x]>25:   # We start after 25 cases has been identified
            half=s[x]/2
            mask=[s < half][0] 
            half_idx = mask.values.argmin()
            frac_day=1-(half-s[half_idx-1])/(s[half_idx]-s[half_idx-1])
            doubling_days=x-half_idx+frac_day
            r[x]=1/doubling_days

    d=r    
    opts = {'ylim':(0,1),'legend': False, 'logy': logy, 'grid': True, 'width': 320, 'height': 200,'padding':0.1,'size':10}
    d.name = name
    lines = d.hvplot(**opts)
    dots = d.hvplot.scatter(**opts)
    hline2=hv.HLine(0.5).opts(color='red', line_width=1,line_dash='solid')
    hline3=hv.HLine(0.33).opts(color='red', line_width=1,line_dash='dashed')
    hline5=hv.HLine(0.2).opts(color='red', line_width=1,line_dash='dashdot')
    hline10=hv.HLine(0.1).opts(color='red', line_width=1,line_dash='dotted')
    text2d = hv.Text(s.index[1],0.53,'2 days',fontsize=8)
    text3d = hv.Text(s.index[1],0.36,'3 days',fontsize=8)
    text5d = hv.Text(s.index[1],0.23,'5 days',fontsize=8)
    text10d = hv.Text(s.index[1],0.13,'10 days',fontsize=8)
    hstate = (lines * dots  *hline2 * hline3 * hline5 *hline10 * text2d * text3d * text5d * text10d).opts(show_legend=True)
    return hstate

ylim=(0,1)
ylabel='Inverse Number of days for COVID-19 Deaths to Double'
title_format='COVID-19 Days for Deaths (>25) to Double: {}'.format(df.columns[-1].date())
ol=[]
for i in countries:
    ol.append(double_rate(i))

overlay=hv.Layout(ol)
overlay.opts(
    merge_tools=True,title=title_format)
overlay.cols(2)


<a id='DoublingDaysConfirmedCasesUS'></a>
## Number of Days to Double Confirmed Cases: U.S. States Comparison
[back to top](#top)

The plots below show the change rate (${doubling.rate}_{confirmed.cases}$) in **number of days for confirmed cases to double** ($days_{confirmed.cases.double}$). This is expressed as

${doubling.rate}_{confirmed.cases} = \frac{1}{days_{confirmed.cases.double}}$

In this representation a factor of 1 means cases double every day, 0.5 means cases double every 2nd day, 0.33 means cases double every third daty, 0.25 menas cases double every 4th day, etc. When the line approaches 0, no more cases are identified.  
Plots begin at **more than 100 confirmed cases**.   
(Plots inspired by Jennifer Bardwell, Jim Bardwell).


In [13]:
df = pd.read_csv(cv19_co,na_values=0)
df.fillna(0,inplace=True)
df.iloc[:,5:] = df.iloc[:,5:].astype(int)

df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))

#df = df[[x in states for x in df.state.values]]

def double_rate(name='Massachusetts',code='MA',logy=False,startdate=dt.datetime(2020,3,7).date()):
    state = df[(df.state==name) | (df.state==code)]
    s = state.select_dtypes(np.int).sum()
    r = pd.Series([np.nan for x in range(len(s.values))],index=s.index)
    for x in range(len(s.values)-1,0,-1):
        if s[x]>100:   # We start after 25 cases has been identified
            half=s[x]/2
            mask=[s < half][0] 
            half_idx = mask.values.argmin()
            frac_day=1-(half-s[half_idx-1])/(s[half_idx]-s[half_idx-1])
            doubling_days=x-half_idx+frac_day
            r[x]=1/doubling_days

    d=r    
    xlim=(startdate,dt.datetime.now().date()+dt.timedelta(1))
    title='{} ({:,})'.format(name,s[-1])
    opts = {'title':title,'xlim':xlim,'ylim':(0,1),'legend': False, 'logy': logy, 'grid': True, 'width': 320, 'height': 200,'padding':0.1,'size':10}
    d.name = name
    lines = d.hvplot(**opts)
    dots = d.hvplot.scatter(**opts)
    hline2=hv.HLine(0.5).opts(color='red', line_width=1,line_dash='solid')
    hline3=hv.HLine(0.33).opts(color='red', line_width=1,line_dash='dashed')
    hline5=hv.HLine(0.2).opts(color='red', line_width=1,line_dash='dashdot')
    hline10=hv.HLine(0.1).opts(color='red', line_width=1,line_dash='dotted')
    text2d = hv.Text(startdate+dt.timedelta(2),0.53,'2 days',fontsize=8)
    text3d = hv.Text(startdate+dt.timedelta(2),0.36,'3 days',fontsize=8)
    text5d = hv.Text(startdate+dt.timedelta(2),0.23,'5 days',fontsize=8)
    text10d = hv.Text(startdate+dt.timedelta(2),0.13,'10 days',fontsize=8)
    hstate = (lines * dots  *hline2 * hline3 * hline5 *hline10 * text2d * text3d * text5d * text10d).opts(show_legend=False)
    return hstate

ylim=(0,1)
ylabel='Inverse Number of days for U.S. COVID-19 Confirmed Cases to Double'
title_format='U.S. COVID-19 Days for Confirmed Cases (>100) to Double: {}'.format(df.columns[-1].date())
ol=[]
for i in states:
    ol.append(double_rate(*i))

overlay=hv.Layout(ol)
overlay.opts(
    merge_tools=True,title=title_format)
overlay.cols(2)


:Layout
   .Overlay.Massachusetts :Overlay
      .Curve.Massachusetts   :Curve   [index]   (Massachusetts)
      .Scatter.Massachusetts :Scatter   [index]   (Massachusetts)
      .HLine.I               :HLine   [x,y]
      .HLine.II              :HLine   [x,y]
      .HLine.III             :HLine   [x,y]
      .HLine.IV              :HLine   [x,y]
      .Text.I                :Text   [x,y]
      .Text.II               :Text   [x,y]
      .Text.III              :Text   [x,y]
      .Text.IV               :Text   [x,y]
   .Overlay.Connecticut   :Overlay
      .Curve.Connecticut   :Curve   [index]   (Connecticut)
      .Scatter.Connecticut :Scatter   [index]   (Connecticut)
      .HLine.I             :HLine   [x,y]
      .HLine.II            :HLine   [x,y]
      .HLine.III           :HLine   [x,y]
      .HLine.IV            :HLine   [x,y]
      .Text.I              :Text   [x,y]
      .Text.II             :Text   [x,y]
      .Text.III            :Text   [x,y]
      .Text.IV             :Text   [x,y]
   .Overlay.Washington    :Overlay
      .Curve.Washington   :Curve   [index]   (Washington)
      .Scatter.Washington :Scatter   [index]   (Washington)
      .HLine.I            :HLine   [x,y]
      .HLine.II           :HLine   [x,y]
      .HLine.III          :HLine   [x,y]
      .HLine.IV           :HLine   [x,y]
      .Text.I             :Text   [x,y]
      .Text.II            :Text   [x,y]
      .Text.III           :Text   [x,y]
      .Text.IV            :Text   [x,y]
   .Overlay.Texas         :Overlay
      .Curve.Texas   :Curve   [index]   (Texas)
      .Scatter.Texas :Scatter   [index]   (Texas)
      .HLine.I       :HLine   [x,y]
      .HLine.II      :HLine   [x,y]
      .HLine.III     :HLine   [x,y]
      .HLine.IV      :HLine   [x,y]
      .Text.I        :Text   [x,y]
      .Text.II       :Text   [x,y]
      .Text.III      :Text   [x,y]
      .Text.IV       :Text   [x,y]
   .Overlay.California    :Overlay
      .Curve.California   :Curve   [index]   (California)
      .Scatter.California :Scatter   [index]   (California)
      .HLine.I            :HLine   [x,y]
      .HLine.II           :HLine   [x,y]
      .HLine.III          :HLine   [x,y]
      .HLine.IV           :HLine   [x,y]
      .Text.I             :Text   [x,y]
      .Text.II            :Text   [x,y]
      .Text.III           :Text   [x,y]
      .Text.IV            :Text   [x,y]
   .Overlay.New_York      :Overlay
      .Curve.New_York   :Curve   [index]   (New York)
      .Scatter.New_York :Scatter   [index]   (New York)
      .HLine.I          :HLine   [x,y]
      .HLine.II         :HLine   [x,y]
      .HLine.III        :HLine   [x,y]
      .HLine.IV         :HLine   [x,y]
      .Text.I           :Text   [x,y]
      .Text.II          :Text   [x,y]
      .Text.III         :Text   [x,y]
      .Text.IV          :Text   [x,y]
   .Overlay.Georgia       :Overlay
      .Curve.Georgia   :Curve   [index]   (Georgia)
      .Scatter.Georgia :Scatter   [index]   (Georgia)
      .HLine.I         :HLine   [x,y]
      .HLine.II        :HLine   [x,y]
      .HLine.III       :HLine   [x,y]
      .HLine.IV        :HLine   [x,y]
      .Text.I          :Text   [x,y]
      .Text.II         :Text   [x,y]
      .Text.III        :Text   [x,y]
      .Text.IV         :Text   [x,y]
   .Overlay.Illinois      :Overlay
      .Curve.Illinois   :Curve   [index]   (Illinois)
      .Scatter.Illinois :Scatter   [index]   (Illinois)
      .HLine.I          :HLine   [x,y]
      .HLine.II         :HLine   [x,y]
      .HLine.III        :HLine   [x,y]
      .HLine.IV         :HLine   [x,y]
      .Text.I           :Text   [x,y]
      .Text.II          :Text   [x,y]
      .Text.III         :Text   [x,y]
      .Text.IV          :Text   [x,y]
   .Overlay.Michigan      :Overlay
      .Curve.Michigan   :Curve   [index]   (Michigan)
      .Scatter.Michigan :Scatter   [index]   (Michigan)
      .HLine.I          :HLine   [x,y]
      .HLine.II         :HLine   [x,y]
      .HLine.III        :HLine   [x,y]
      .HLi

<a id='DoublingDaysDeathsUS'></a>
## Number of Days to Double Deaths: U.S. States Comparison
[back to top](#top)

The plots below show the change rate (${doubling.rate}_{deaths}$) in **number of days for deaths to double** ($days_{deaths.double}$). This is expressed as

${doubling.rate}_{deaths} = \frac{1}{days_{deaths.double}}$

In this representation a factor of 1 means death counts double every day, 0.5 means death counts double every 2nd day, 0.33 means death counts double every third day, 0.25 menas death counts double every 4th day, etc. When the line approaches 0, no more deaths are counted.  
Plots begin at **more than 10 deaths**.  
*(Plots inspired by Jennifer Bardwell, Jim Bardwell).*


In [49]:
df = pd.read_csv(cv19_do,na_values=0)
df.fillna(0,inplace=True)
df.iloc[:,5:] = df.iloc[:,5:].astype(int)

df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))

#df = df[[x in states for x in df.state.values]]

def double_rate(name='Massachusetts',code='MA',logy=False,startdate=dt.datetime(2020,3,7).date()):
    state = df[(df.state==name) | (df.state==code)]
    s = state.select_dtypes(np.int).sum()
    r = pd.Series([np.nan for x in range(len(s.values))],index=s.index)
    for x in range(len(s.values)-1,0,-1):
        if s[x]>10:   # We start after 25 cases has been identified
            half=s[x]/2
            mask=[s < half][0] 
            half_idx = mask.values.argmin()
            frac_day=1-(half-s[half_idx-1])/(s[half_idx]-s[half_idx-1])
            doubling_days=x-half_idx+frac_day
            r[x]=1/doubling_days

    d=r    
    xlim=(startdate,dt.datetime.now().date()+dt.timedelta(1))
    opts = {'xlim':xlim,'ylim':(0,1),'legend': False, 'logy': logy, 'grid': True, 'width': 320, 'height': 200,'padding':0.1,'size':10}
    d.name = name
    lines = d.hvplot(**opts)
    dots = d.hvplot.scatter(**opts)
    hline2=hv.HLine(0.5).opts(color='red', line_width=1,line_dash='solid')
    hline3=hv.HLine(0.33).opts(color='red', line_width=1,line_dash='dashed')
    hline5=hv.HLine(0.2).opts(color='red', line_width=1,line_dash='dashdot')
    hline10=hv.HLine(0.1).opts(color='red', line_width=1,line_dash='dotted')
    text2d = hv.Text(startdate+dt.timedelta(2),0.53,'2 days',fontsize=8)
    text3d = hv.Text(startdate+dt.timedelta(2),0.36,'3 days',fontsize=8)
    text5d = hv.Text(startdate+dt.timedelta(2),0.23,'5 days',fontsize=8)
    text10d = hv.Text(startdate+dt.timedelta(2),0.13,'10 days',fontsize=8)
    hstate = (lines * dots  *hline2 * hline3 * hline5 *hline10 * text2d * text3d * text5d * text10d).opts(show_legend=True)
    return hstate

ylim=(0,1)
ylabel='Inverse Number of days for U.S. COVID-19 Deaths to Double'
title_format='U.S. COVID-19 Days for Deaths (>10) to Double: {}'.format(df.columns[-1].date())
ol=[]
for i in states:
    ol.append(double_rate(*i))

overlay=hv.Layout(ol)
overlay.opts(
    merge_tools=True,title=title_format)
overlay.cols(2)


:Layout
   .Overlay.Massachusetts :Overlay
      .Curve.Massachusetts   :Curve   [index]   (Massachusetts)
      .Scatter.Massachusetts :Scatter   [index]   (Massachusetts)
      .HLine.I               :HLine   [x,y]
      .HLine.II              :HLine   [x,y]
      .HLine.III             :HLine   [x,y]
      .HLine.IV              :HLine   [x,y]
      .Text.I                :Text   [x,y]
      .Text.II               :Text   [x,y]
      .Text.III              :Text   [x,y]
      .Text.IV               :Text   [x,y]
   .Overlay.Connecticut   :Overlay
      .Curve.Connecticut   :Curve   [index]   (Connecticut)
      .Scatter.Connecticut :Scatter   [index]   (Connecticut)
      .HLine.I             :HLine   [x,y]
      .HLine.II            :HLine   [x,y]
      .HLine.III           :HLine   [x,y]
      .HLine.IV            :HLine   [x,y]
      .Text.I              :Text   [x,y]
      .Text.II             :Text   [x,y]
      .Text.III            :Text   [x,y]
      .Text.IV             :Text   [x,y]
   .Overlay.Washington    :Overlay
      .Curve.Washington   :Curve   [index]   (Washington)
      .Scatter.Washington :Scatter   [index]   (Washington)
      .HLine.I            :HLine   [x,y]
      .HLine.II           :HLine   [x,y]
      .HLine.III          :HLine   [x,y]
      .HLine.IV           :HLine   [x,y]
      .Text.I             :Text   [x,y]
      .Text.II            :Text   [x,y]
      .Text.III           :Text   [x,y]
      .Text.IV            :Text   [x,y]
   .Overlay.Texas         :Overlay
      .Curve.Texas   :Curve   [index]   (Texas)
      .Scatter.Texas :Scatter   [index]   (Texas)
      .HLine.I       :HLine   [x,y]
      .HLine.II      :HLine   [x,y]
      .HLine.III     :HLine   [x,y]
      .HLine.IV      :HLine   [x,y]
      .Text.I        :Text   [x,y]
      .Text.II       :Text   [x,y]
      .Text.III      :Text   [x,y]
      .Text.IV       :Text   [x,y]
   .Overlay.California    :Overlay
      .Curve.California   :Curve   [index]   (California)
      .Scatter.California :Scatter   [index]   (California)
      .HLine.I            :HLine   [x,y]
      .HLine.II           :HLine   [x,y]
      .HLine.III          :HLine   [x,y]
      .HLine.IV           :HLine   [x,y]
      .Text.I             :Text   [x,y]
      .Text.II            :Text   [x,y]
      .Text.III           :Text   [x,y]
      .Text.IV            :Text   [x,y]
   .Overlay.New_York      :Overlay
      .Curve.New_York   :Curve   [index]   (New York)
      .Scatter.New_York :Scatter   [index]   (New York)
      .HLine.I          :HLine   [x,y]
      .HLine.II         :HLine   [x,y]
      .HLine.III        :HLine   [x,y]
      .HLine.IV         :HLine   [x,y]
      .Text.I           :Text   [x,y]
      .Text.II          :Text   [x,y]
      .Text.III         :Text   [x,y]
      .Text.IV          :Text   [x,y]
   .Overlay.Georgia       :Overlay
      .Curve.Georgia   :Curve   [index]   (Georgia)
      .Scatter.Georgia :Scatter   [index]   (Georgia)
      .HLine.I         :HLine   [x,y]
      .HLine.II        :HLine   [x,y]
      .HLine.III       :HLine   [x,y]
      .HLine.IV        :HLine   [x,y]
      .Text.I          :Text   [x,y]
      .Text.II         :Text   [x,y]
      .Text.III        :Text   [x,y]
      .Text.IV         :Text   [x,y]
   .Overlay.Illinois      :Overlay
      .Curve.Illinois   :Curve   [index]   (Illinois)
      .Scatter.Illinois :Scatter   [index]   (Illinois)
      .HLine.I          :HLine   [x,y]
      .HLine.II         :HLine   [x,y]
      .HLine.III        :HLine   [x,y]
      .HLine.IV         :HLine   [x,y]
      .Text.I           :Text   [x,y]
      .Text.II          :Text   [x,y]
      .Text.III         :Text   [x,y]
      .Text.IV          :Text   [x,y]
   .Overlay.Michigan      :Overlay
      .Curve.Michigan   :Curve   [index]   (Michigan)
      .Scatter.Michigan :Scatter   [index]   (Michigan)
      .HLine.I          :HLine   [x,y]
      .HLine.II         :HLine   [x,y]
      .HLine.III        :HLine   [x,y]
      .HLi

<a id='Reading'></a>

# We hope these data are informative and convey how seriously we have to take the COVID-19 pandemic. Stay safe.


## Suggested Reading:
[back to top](#top)


[Coronavirus Interview with Larry Brilliant](https://www.wired.com/story/coronavirus-interview-larry-brilliant-smallpox-epidemiologist/?utm_source=pocket-newtab)

[Brookings Institute: A mortality perspective on COVID-19: Time, location, and age](https://www.brookings.edu/blog/future-development/2020/03/23/a-mortality-perspective-on-covid-19-time-location-and-age/)



<a id='DataSources'></a>
## Data Sources
[back to top](#top)
### Johns Hopkins University daily updated COVID-19 data
COVID-19 confirmed cases, deaths and recovered cases data are streamed from the [The Center for Systems Science and Engineering (CSSE)](https://systems.jhu.edu) at Johns Hopkins University. The CCSE COVID-19 [GitHub Repo](https://github.com/CSSEGISandData/COVID-19) has more information about these data and their sources.

Since Johns Hopkins changed the data format on 2020-03-23, they do not provide the US compiled time series data yet. We acknowldege Sooth Sawyer who helped out by compiling the data set in the old format at:
[https://www.soothsawyer.com](https://www.soothsawyer.com/john-hopkins-time-series-data-confirmed-case-csv-after-march-22-2020/?github=1)

### UN Population Data
We obtain the Population data from UN statistics.  [UN Population Data Sets](https://population.un.org/wpp/Download/Standard/Population/) have more information about these data and their sources.

### US Population Data
US population data ar obtained from US Census statistics.  [US Population Data Sets](http://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv?#) have more information about these data and their sources.